In [4]:
import numpy as np
import scipy as sp

In [56]:
def SPCA(C, k):
    omega = np.random.randn(C.shape[1], k)
    
    print ("gauss random matrix shape ", omega.shape)
    
    _C = np.zeros(C.shape)
    # Trimming
    for i in range (C.shape[0]):
        if (np.count_nonzero(C[i,:]) > 10):
            _C[i,:] = 0
        else:
            _C[i,:] = C[i,:]
    print ("_C shape ", _C.shape)        
    
    F = _C.T @ _C - np.diag(np.diag( _C.T @ _C))
    
    print ("F shape ", F.shape)
    
    QR = (np.linalg.matrix_power(F, int(5 * np.log2(C.shape[1]))) @ omega)
    
    print ("QR shape ", QR.shape)
    
    ## TODO: power iteration ???
    ## Is log2 ???
    Q, R = np.linalg.qr(QR)
    
    return Q

In [101]:
def SLA(M, k, delta, l):
    
    print ("k ", k)
    print ("l ", l)
    print ("M shape ", M.shape)
    
    
    #l_samples = int (np.random.choice(M.shape[1], l))
    l_samples = np.random.randint(low = 0, high = M.shape[1], size = l)
    
    print ("sample len ", len(l_samples))
        
    A_b1 = A_b2 = M[:, list(l_samples)]
    
    print ("A_b1 shape ", A_b1.shape)
    
    
    _M = np.delete(M, list(l_samples), 1)
    
    Q = SPCA(A_b1, k)
    
    print ("Q shape ", Q.shape)
    
    for i in range (A_b2.shape[0]):
        if (np.count_nonzero(A_b2[i,:]) > 2):
            A_b2[i,:] = 0
    for i in range (A_b2.shape[1]):
        if (np.count_nonzero(A_b2[:,i]) > 10 * delta * A_b2.shape[0]):
            A_b2[:,i] = 0
    
    W = A_b2 @ Q
    V = np.zeros((M.shape[1], k))
    
    print (A_b1.shape)
    print (W.shape)
    print ((V[0:l,:]).shape)
    
    V[0:l,:] = (A_b1).T @ W
    
    ## FIXME: ??? what size ???
    I = np.zeros((M.shape[0], k))
    I = A_b1 @ V[0:l,:]
    
    ### TODO: Need to remove A_b1, A_b2 and Q from RAM
    
    for t in range (M.shape[1] - l):
        A_t = _M[:, t]
        V[t, :] = (A_t.T) @ W
        I += (A_t.reshape(-1, 1)) @ V[t,:].reshape(-1, 1).T
        ## TODO: remove A_t from RAM ???
    
    Q, R = np.linalg.qr(V)
    U = (1/delta)*(I @ R @ R.T)
    
    ## FIXME: Need to find out what is here in actual algorithm
    M_k = U @ V.T
    print (M_k.shape)
    
    return M_k
        

In [102]:
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import requests

url = 'https://cdn.pixabay.com/photo/2013/06/29/21/18/wolf-142173_1280.jpg'

image_raw = Image.open(requests.get(url, stream=True).raw).convert("L")
image = np.array(image_raw)
print (image.shape)

(853, 1280)


In [103]:
l = int(image.shape[1]/10)
#print (l)
delta = 1/(l * np.log2(image.shape[0]))
M = SLA(image, int (np.linalg.matrix_rank(image) / 10), delta, l)

k  85
l  128
M shape  (853, 1280)
sample len  128
A_b1 shape  (853, 128)
gauss random matrix shape  (128, 85)
_C shape  (853, 128)
F shape  (128, 128)
QR shape  (128, 85)
Shape of my heart  (128, 85)
Q shape  (128, 85)
(853, 128)
(853, 85)
(128, 85)
(853, 1280)


In [104]:
print (np.linalg.matrix_rank(M))

0
